In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### iteration시마다 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [ ]:
!pip install scikit-learn==1.0.2

In [2]:
import sklearn

print(sklearn.__version__)

1.0.2


### 1번 - 보스턴 데이터셋을 불러오겠다

In [3]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston

boston = load_boston()  #Boston Housing Dataset을 로드하고, 이를 boston 변수에 저장합니다.
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
#boston.data는 데이터셋의 특성(features) 값들을 포함하는 NumPy 배열입니다.
#boston.feature_names는 각 특성의 이름을 포함하는 리스트입니다.
#pd.DataFrame은 pandas DataFrame 객체를 생성하는 함수입니다.
#여기서는 boston.data를 사용해 DataFrame을 만들고, 각 열(column)에 boston.feature_names에 있는 이름을 할당합니다.
bostonDF['PRICE'] = boston.target
#boston.target은 데이터셋의 타겟 값(즉, 주택 가격)을 포함하는 NumPy 배열입니다.
#bostonDF['PRICE']는 DataFrame에 'PRICE'라는 이름의 새로운 열(column)을 추가하고, 여기에 boston.target의 값을 할당합니다.
print(bostonDF.shape)
bostonDF.head()

(506, 14)


/opt/conda/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### 2번 - 가중치(whight), 절편(bias)을 업데이트 하기 위한 함수를 선언하겠다.

In [4]:
#가중치(weight)와 절편(bias)을 업데이트하는 함수를 생성
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    # 데이터 건수
    N = target_batch.shape[0] #target_batch 행렬의 행의 갯수를 리턴 (열의 갯수는 .shape[1])
    # 예측 값. = 가중치1*피쳐1 + 가중치2*피쳐2 + 절편
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    # 실제값과 예측값의 차이 
    diff_batch = target_batch - predicted_batch
    
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    #rm_batch 행렬은 n*1 배열이므로 .T를 활용해 1*n 배열로 변환해준다.
    #그리고 n*1 배열인 diff_batch과 백터내적 계산(np.dot)을 수행한다. (w1*x1 + w2*x2)
    #거기에 -(2/N)*learning_rate를 곱해준다. 그러면 업데이트가 완료됐다.
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

### 3번 - 정해진 반복(iteration) 수만큼 손실함수 편미분을 구하고
###      가중치(whight), 절편(bias) 업데이트 함수(2번)을 사용해서
###      업데이트까지 수행하는 함수를 선언하겠다.

In [13]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
# 앞으로 들어올 인자
#   batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, batch_size=30, verbose=True)
def batch_random_gradient_descent(features, target, iter_epochs, batch_size, verbose): #batch_size가 추가됨.
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed(2021) #난수생성을 위한 시드 설정. 시드를 2021 등으로 고정하면 난수가 항상 같게나온다. 데이터 검증 일관성을 위해 설정.
    w1 = np.zeros((1,)) #초기값 설정을 위해 w1에 0행렬을 만들어준다. (1,)이므로 dataframe이 아닌 Series [0.] 이다.
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    #함수의 features 인자로 scaled_features 값이 들어옴. scaled_features는 rm과 lstat으로 이루어진 506,2 배열임.
    rm = features[:, 0] #features의 1열 모든 행에 대한 배열을 rm에 저장하겠다.(방의 갯수) 506개
    lstat = features[:, 1] #features의 2열 모든 행에 대한 배열을 lstat에 저장하겠다.(하위계층) 506개
    #결론은 n행 2열의 데이터를 n행 1열 데이터로 찢겠다는것임.

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 갯수만큼 데이터를 임의로 선택.
        # 여기서는 PRICE 열의 차원의 크기(506개) 중 batch_size(30개)의 데이터를 랜덤으로 추출해서 batch_indexes에 저장하겠다는 뜻임.
        # 핵심은 506개 중에 30개 랜덤으로 뽑는 함수를 만들었다고 생각하면 됨.
        batch_indexes = np.random.choice(target.shape[0], batch_size) #SGD에서는 1이었는데 Mini-Batch에서는 batch_size만큼
        rm_batch = rm[batch_indexes] #rm배열(features 인자로 들어오는 배열의 1열) 데이터를 랜덤으로 batch_size만큼 선별해서 rm_batch에 저장.
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        # Batch GD 기반으로 Weight/Bias의 Update를 구함. 
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        # Batch GD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

### 참고 - 배열의 크기를 구하는 법(len, shape) / ramdom.choice 사용법 / batch_indexes 사용법(코드실행x)

In [10]:
print(len(bostonDF['RM'])) #boston Dataframe의 RM 이라는 열에 대한 크기를 반환하겠다.(len)
print(bostonDF['RM'].shape[0]) #boston Dataframe의 RM 이라는 열에 대해 차원의 크기를 반환하겠다.[0]이면 행의 갯수를 반환 / [1]은 열의 갯수

#batch_indexes = 506개의 데이터 중 무작위로 30개를 추출해서 np 배열로 나타내겠다.
batch_indexes = np.random.choice(len(bostonDF['RM']), 30)
#batch_indexes를 실행해서 결과를 표시해라.
print(batch_indexes)

#boston 데이터 프레임의 RM이라는 이름을 가진 열 데이터. 의 값(values)을 기준으로 batch_indexes(무작위로 30개)로 선별하겠다.
bostonDF['RM'].values[batch_indexes]
print(bostonDF['RM'].values[batch_indexes])


506
506
[430 404 166  29 172 241 389 379 332 326  29 204 211 396 338 261 103   1
 380 166 390  99 347 241 217 426 465  14 188 133]
[6.348 5.531 7.929 6.674 5.572 6.095 5.39  6.223 6.031 6.312 6.674 8.034
 5.404 6.405 6.059 7.52  6.137 6.421 6.968 7.929 5.713 7.416 6.516 6.095
 6.642 5.837 5.759 6.096 6.556 5.822]


### 4번 - 위에 선언한 함수를 사용해서 본격적으로 Gradient Descent를 수행하겠다.
* 데이터셋에서 모든 features를 다 사용하지 않고 rm, lstat만 사용할 것이므로 스케일을 줄인 scaled_features를 만든다.
* 신경망 연산을 위해 데이터 정규화/표준화 작업을 선행해야한다.
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환(fit_transform)후 학습 적용.

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])
print(scaled_features[:15])

[[0.57750527 0.08967991]
 [0.5479977  0.2044702 ]
 [0.6943859  0.06346578]
 [0.65855528 0.03338852]
 [0.68710481 0.09933775]
 [0.54972217 0.09602649]
 [0.4696302  0.29525386]
 [0.50028741 0.48068433]
 [0.39662771 0.7781457 ]
 [0.46809734 0.424117  ]
 [0.53956697 0.51655629]
 [0.46905537 0.31843267]
 [0.44606246 0.38576159]
 [0.45755892 0.18018764]
 [0.48572523 0.23537528]]


### 5번 - batch_random_gradient_descent 함수를 실행한다.
* (4번에서 스케일링한 scaled_features, target, 1000번 연산반복, 30개 랜덤추출, log=true)

In [15]:
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000
w1: [0.27876422] w2: [0.09698635] bias: [0.46993333] loss: 563.291813385093
Epoch: 2 / 1000
w1: [0.47886984] w2: [0.20026232] bias: [0.88288155] loss: 539.5145044664642
Epoch: 3 / 1000
w1: [0.64939428] w2: [0.30944032] bias: [1.22380595] loss: 519.976239745994
Epoch: 4 / 1000
w1: [0.90962021] w2: [0.40653277] bias: [1.66852375] loss: 494.70368515773123
Epoch: 5 / 1000
w1: [1.09334893] w2: [0.51275835] bias: [2.01825894] loss: 475.54326873700523
Epoch: 6 / 1000
w1: [1.29981565] w2: [0.60425017] bias: [2.4072638] loss: 454.9607353035734
Epoch: 7 / 1000
w1: [1.53787989] w2: [0.68467221] bias: [2.80537192] loss: 434.0124430042986
Epoch: 8 / 1000
w1: [1.78198857] w2: [0.77121673] bias: [3.21527466] loss: 413.05907884860613
Epoch: 9 / 1000
w1: [1.97775789] w2: [0.84818678] bias: [3.56665015] loss: 395.85302672810496
Epoch: 10 / 1000
w1: [2.21299523] w2: [0.92982135] bias: [3.96558641] loss: 376.6605838772106
Epoch: 11 / 1000
w1: [2.43283837] w2

### 6번 - 경사하강법을 적용해서 나온 최종 w1, w2, bias 값으로 실제 집값과 예측값을 비교한다.

In [16]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_BATCH_RANDOM
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.257214
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.223492
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.796757
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.508907
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.197770
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.652067
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.578291
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.768683
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,13.986236
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.890450


### iteration 시에 순차적으로 일정한 batch 크기만큼의 데이터를 전체 학습데이터에 걸쳐서 가져오는 Mini-Batch GD 수행

In [ ]:

for batch_step in range(0, 506, 30):
    print(batch_step)

In [17]:
bostonDF['PRICE'].values[480:510]

array([23. , 23.7, 25. , 21.8, 20.6, 21.2, 19.1, 20.6, 15.2,  7. ,  8.1,
       13.6, 20.1, 21.8, 24.5, 23.1, 19.7, 18.3, 21.2, 17.5, 16.8, 22.4,
       20.6, 23.9, 22. , 11.9])

In [18]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size): #(start, end, step)
            # batch_size만큼 순차적인 데이터를 가져옴. 
            # batch_size를 키우면 한번에 가져오는 데이터 양이 많아지는 만큼 한번에 처리하는 데이터 양도 많아진다.
            # 다만, 그만큼 메모리가 많이 필요하게 돼서 잘못하면 컴퓨터가 뻗는 문제가 있다.
            rm_batch = rm[batch_step:batch_step + batch_size] #최초 0:30 / 다음 30:60 / 다음 60:90 / ...
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
        
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

            # Batch GD로 구한 weight/bias의 update 적용. 
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
        
            if verbose:
                print('Epoch:', i+1,'/', iter_epochs, 'batch step:', batch_step)
                # Loss는 전체 학습 데이터 기반으로 구해야 함.
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [22]:
w1, w2, bias = batch_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000 batch step: 0
w1: [0.21118173] w2: [0.11625443] bias: [0.41353333] loss: 567.2250927629614
Epoch: 1 / 1000 batch step: 30
w1: [0.42504999] w2: [0.21151116] bias: [0.83382878] loss: 542.7895112848258
Epoch: 1 / 1000 batch step: 60
w1: [0.65529552] w2: [0.29207843] bias: [1.26864819] loss: 518.1392189598428
Epoch: 1 / 1000 batch step: 90
w1: [0.89614794] w2: [0.39426111] bias: [1.69338585] loss: 494.12115632033255
Epoch: 1 / 1000 batch step: 120
w1: [1.02658639] w2: [0.5371618] bias: [1.98799585] loss: 478.0108246502634
Epoch: 1 / 1000 batch step: 150
w1: [1.3122922] w2: [0.6192097] bias: [2.47174746] loss: 452.07615796064226
Epoch: 1 / 1000 batch step: 180
w1: [1.70976219] w2: [0.69994868] bias: [3.0759416] loss: 420.237402226281
Epoch: 1 / 1000 batch step: 210
w1: [2.04599772] w2: [0.79636541] bias: [3.5818841] loss: 394.35643754954594
Epoch: 1 / 1000 batch step: 240
w1: [2.39049798] w2: [0.87851357] bias: [4.08502278] loss: 369.530587101

In [23]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_BATCH_RANDOM,PREDICTED_PRICE_BATCH
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.257214,28.830077
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.223492,25.370027
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.796757,32.508278
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.508907,32.272631
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.197770,31.477882
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.652067,27.953052
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.578291,21.190621
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.768683,17.655029
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,13.986236,7.975749
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.890450,18.134764


### Mini BATCH GD를 Keras로 수행
* Keras는 기본적으로 Mini Batch GD를 수행
* Keras는 기본적으로 batch_size를 32로 줌

In [24]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

# Keras는 반드시 Batch GD를 적용함. batch_size가 None이면 32를 할당. 
model.fit(scaled_features, bostonDF['PRICE'].values, batch_size=30, epochs=1000)

2024-05-18 08:54:15.337052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 08:54:15.337157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 08:54:15.339192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/1000


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 525.9944 - mse: 525.9944   
Epoch 2/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 519.0429 - mse: 519.0429 
Epoch 3/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 536.8857 - mse: 536.8857 
Epoch 4/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 508.7723 - mse: 508.7723 
Epoch 5/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 495.2387 - mse: 495.2387 
Epoch 6/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 465.5489 - mse: 465.5489 
Epoch 7/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 489.2552 - mse: 489.2552 
Epoch 8/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 424.0217 - mse: 424.0217 
Epoch 9/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 444.8386 - mse: 444.8386 
Epoch 10/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 423.0359 - mse: 423.0359 
Epoch 11/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 419.3540 - mse: 419.3540 
Epoch 12/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 41

In [25]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_BATCH_RANDOM,PREDICTED_PRICE_BATCH,KERAS_PREDICTED_PRICE_BATCH
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.257214,28.830077,28.951946
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.223492,25.370027,25.475517
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.796757,32.508278,32.615044
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.508907,32.272631,32.391144
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.197770,31.477882,31.579201
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.652067,27.953052,28.078491
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.578291,21.190621,21.292572
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.768683,17.655029,17.717123
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,13.986236,7.975749,8.000051
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.890450,18.134764,18.212921
